Trainer()-> High-level class to fine-tune models easily

TrainingArguments()-> Defines how your model is trained: batch size, epochs, logging

tokenizer.map()-> Prepares data for the model by tokenizing it

from_pretrained()->	Loads the pretrained weights

save_pretrained()-> Saves your model for deployment

In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system =

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import torch

In [3]:
model_name="distilbert-base-uncased"
tokenizer=AutoTokenizer.from_pretrained(model_name)
model=AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

#num_labels=2 -> for binary classification (pos & neg)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
data=load_dataset("imdb")  # IMDb-> dataset for sentiment classification, already splitted into training and testing set

README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [5]:
import pandas as pd
df=pd.DataFrame(data["train"])
df.head()

,text,label
0,I rented I AM CURIOUS-YELLOW from my video sto...,0
1,"""I Am Curious: Yellow"" is a risible and preten...",0
2,If only to avoid making this type of film in t...,0
3,This film was probably inspired by Godard's Ma...,0
4,"Oh, brother...after hearing about this ridicul...",0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    25000 non-null  object
 1   label   25000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 390.8+ KB


In [7]:
# preprocess the dataset
def preprocess(text):
  return tokenizer(text["text"], truncation=True, padding="max_length", max_length=512)

# truncation -> cut off long text to max length
# padding-> add padding so all texts have same length
# max_length=512 -> max length supported by BERT

encoded=data.map(preprocess, batched= True)
encoded.set_format("torch", columns=['input_ids', 'attention_mask', 'label'])

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [8]:
# define training arguments
training_args=TrainingArguments(
    output_dir="./results",      # where to save trained model (model checkpoints)
    evaluation_strategy="epoch",  # evaluate model at the end of each epoch
    save_strategy="epoch",        # save model at the end of each epoch
    learning_rate=2e-5,           # controls how fast model learn
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,           # train over data twice
    weight_decay=0.01,            # regularization to prevent overfitting

)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [9]:
# create the trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded["train"].shuffle(seed=42).select(range(2000)),
    eval_dataset=encoded["test"].shuffle(seed=42).select(range(500)),
)


training_args: instance of TrainingArgumnets that contains all training config like batch size, learning rate, etc.

train_dataset: Randomly shuffle the dataset and pick the first 2000 shuffled samples to speed up training for experimentation

eval_dataset: pick shuffled 500 test samples to evaluate model

In [10]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jyotsanashyama (jyotsanashyama-lovely-professional-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,No log,0.321754
2,0.316900,0.371584


TrainOutput(global_step=500, training_loss=0.31689187622070314, metrics={'train_runtime': 496.7557, 'train_samples_per_second': 8.052, 'train_steps_per_second': 1.007, 'total_flos': 529869594624000.0, 'train_loss': 0.31689187622070314, 'epoch': 2.0})

In [11]:
result=trainer.evaluate()
print(result)

{'eval_loss': 0.3715842366218567, 'eval_runtime': 7.4439, 'eval_samples_per_second': 67.169, 'eval_steps_per_second': 8.463, 'epoch': 2.0}


eval_loss: loss of test(evaluation) set
-> lower means better

In [12]:
model.save_pretrained("my-fine-tuned-model")
tokenizer.save_pretrained("my-fine-tuned-model")

('my-fine-tuned-model/tokenizer_config.json',
 'my-fine-tuned-model/special_tokens_map.json',
 'my-fine-tuned-model/vocab.txt',
 'my-fine-tuned-model/added_tokens.json',
 'my-fine-tuned-model/tokenizer.json')

In [16]:
# making predictions
from transformers import pipeline

model_path="my-fine-tuned-model"
tokenizer=AutoTokenizer.from_pretrained(model_path)
model=AutoModelForSequenceClassification.from_pretrained(model_path)


sent_model=pipeline("text-classification", model=model, tokenizer=tokenizer)
sent_model("I hate this movie!")

Device set to use cuda:0


[{'label': 'LABEL_0', 'score': 0.8582329154014587}]